

*   Step 1 : upload zip file from folders upload in left.
*   Step 2 : upload the annnation file. go to step 3 only after both these files are totally uploaded.


*   Step 3 : enter name of zip in the below subsetname variable only the name (for example if zip is training1.zip subsetname='training1')









In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
subsetpath='/content/drive/MyDrive/val_data/4val.zip'
subsetname='4val'
savename = f'/content/drive/MyDrive/new_dats_val/val_set_{subsetname}.dat'
zipspath=f'/content/dataset'
transcriptionFilePath = '/content/drive/MyDrive/annots/transcriptions.pkl'

In [ ]:
!pip install ffmpeg-python


In [ ]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import ffmpeg as ff
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
import pickle
import datetime
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
# Load BERT tokenizer & model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Load transcriptions
with open(transcriptionFilePath, "rb") as f:
    transcriptions = pickle.load(f)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def extract_audio_from_video(file_path: str) -> np.ndarray:
    inputfile = ff.input(file_path)
    out = inputfile.output('-', format='f32le', acodec='pcm_f32le', ac=1, ar='44100')
    try:
        raw = out.run(capture_stdout=True, capture_stderr=True)
    except ff.Error as e:
        print(e.stderr.decode())
        raise
    return np.frombuffer(raw[0], np.float32)

def preprocess_audio_series(raw_data: np.ndarray) -> np.ndarray:
    N, M = 24, 1319
    mfcc_data = librosa.feature.mfcc(y=raw_data, sr=44100, n_mfcc=24)
    mfcc_data_standardized = (mfcc_data - np.mean(mfcc_data)) / np.std(mfcc_data)
    number_of_columns_to_fill = M - mfcc_data_standardized.shape[1]
    padding = np.zeros((N, number_of_columns_to_fill))
    padded_data = np.hstack((padding, mfcc_data_standardized))
    return padded_data.reshape(N, M, 1)

def get_number_of_frames(file_path: str) -> int:
    probe = ff.probe(file_path)
    video_streams = [stream for stream in probe["streams"] if stream["codec_type"] == "video"]
    return int(video_streams[0]['nb_frames'])

def extract_N_video_frames(file_path: str, number_of_samples: int = 6) -> List[np.ndarray]:
    nb_frames = get_number_of_frames(file_path)
    video_frames = []
    random_indexes = random.sample(range(0, nb_frames), number_of_samples)
    cap = cv2.VideoCapture(file_path)
    for ind in random_indexes:
        cap.set(1, ind)
        res, frame = cap.read()
        video_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    return video_frames

def resize_image(image: np.ndarray, new_size: Tuple[int, int]) -> np.ndarray:
    return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

def crop_image_window(image: np.ndarray, training: bool = True) -> np.ndarray:
    height, width, _ = image.shape
    if training:
        MAX_N = height - 128
        MAX_M = width - 128
        rand_N_index, rand_M_index = random.randint(0, MAX_N), random.randint(0, MAX_M)
        return image[rand_N_index:(rand_N_index+128), rand_M_index:(rand_M_index+128), :]
    else:
        N_index = (height - 128) // 2
        M_index = (width - 128) // 2
        return image[N_index:(N_index+128), M_index:(M_index+128), :]

def reading_label_data(file_name: str, dictionary: Dict[str, str]) -> np.ndarray:
    features = ['extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness']
    extracted_data = [float(dictionary[label][file_name]) for label in features]
    return np.stack(extracted_data).reshape(5, 1)

def get_text_embeddings(text: str) -> np.ndarray:
    if not text:
        return np.zeros((768,))
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = bert_model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def preprocessing_input(file_path: str, file_name: str, dictionary: Dict[str, str], training: bool = True) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    extracted_audio_raw = extract_audio_from_video(file_path)
    preprocessed_audio = preprocess_audio_series(raw_data=extracted_audio_raw)
    sampled = extract_N_video_frames(file_path, number_of_samples=6)
    resized_images = [resize_image(image=im, new_size=(248,140)) for im in sampled]
    cropped_images = [crop_image_window(image=resi, training=training) / 255.0 for resi in resized_images]
    preprocessed_video = np.stack(cropped_images)
    text_transcription = transcriptions.get(file_name, "")
    text_embedding = get_text_embeddings(text_transcription)
    video_gt = reading_label_data(file_name=file_name, dictionary=dictionary)
    return (preprocessed_audio, preprocessed_video, text_embedding, video_gt)


In [ ]:
!unzip {subsetpath} -d /content/dataset

Archive:  /content/drive/MyDrive/val_data/4val.zip
  inflating: /content/dataset/validation80_16.zip  
  inflating: /content/dataset/validation80_17.zip  
  inflating: /content/dataset/validation80_18.zip  
  inflating: /content/dataset/validation80_19.zip  
  inflating: /content/dataset/validation80_20.zip  


In [ ]:
for filename in os.listdir(zipspath):
    !unzip "{zipspath}/{filename}" -d /content/dataset

Archive:  /content/dataset/validation80_17.zip
 extracting: /content/dataset/176vWywoq9E.003.mp4  
 extracting: /content/dataset/Gtq5nfRlRx8.005.mp4  
 extracting: /content/dataset/2HU92Hfx8Lo.000.mp4  
 extracting: /content/dataset/Y1LrgyCJtpU.004.mp4  
 extracting: /content/dataset/xrxAyO8pKUo.000.mp4  
 extracting: /content/dataset/WwmWN63mkT4.000.mp4  
 extracting: /content/dataset/ymrFbEMNBME.002.mp4  
 extracting: /content/dataset/1zpucNpIDk8.000.mp4  
 extracting: /content/dataset/CGfr9jl8w4Y.000.mp4  
 extracting: /content/dataset/sx57p5HyekI.001.mp4  
 extracting: /content/dataset/oU7zAWm_01o.005.mp4  
 extracting: /content/dataset/mIQnQ8Nmj-c.000.mp4  
 extracting: /content/dataset/ZqbJIM7rmO8.004.mp4  
 extracting: /content/dataset/4nMstrTuXXw.000.mp4  
 extracting: /content/dataset/V8w2Lo5wxl8.005.mp4  
 extracting: /content/dataset/jKHaHWuaU14.003.mp4  
 extracting: /content/dataset/8OckVmEZ8X8.005.mp4  
 extracting: /content/dataset/0G9vplL8ae8.004.mp4  
 extracting: /con

In [ ]:

training_set_data = []
path = '/content/dataset'
gt = pickle.load( open( "/content/drive/MyDrive/annots/annotation_validation.pkl", "rb" ), encoding='latin1' )
t1 = datetime.datetime.utcnow()
for filename in os.listdir(path):
    if filename.endswith('.mp4'):
      filePath = path+'/'+filename
      training_set_data.append(preprocessing_input(file_path=filePath, file_name=filename, dictionary=gt, training=True))
t2 = datetime.datetime.utcnow()
#Measuring execution time
print('Elapsed time: ' + str(t2-t1))


'\ntraining_set_data = []\npath = \'/content/dataset\'\ngt = pickle.load( open( "/content/drive/MyDrive/annots/annotation_validation.pkl", "rb" ), encoding=\'latin1\' )\nt1 = datetime.datetime.utcnow()\nfor filename in os.listdir(path):\n    if filename.endswith(\'.mp4\'):\n      filePath = path+\'/\'+filename\n      training_set_data.append(preprocessing_input(file_path=filePath, file_name=filename, dictionary=gt, training=True))\nt2 = datetime.datetime.utcnow()\n#Measuring execution time\nprint(\'Elapsed time: \' + str(t2-t1))\n'

In [ ]:

validation_set_data = []
path = '/content/dataset'  # Change to validation set path
gt = pickle.load(open("/content/drive/MyDrive/annots/annotation_validation.pkl", "rb"), encoding='latin1')
t1 = datetime.datetime.utcnow()

for filename in os.listdir(path):
    if filename.endswith('.mp4'):
        filePath = path + '/' + filename
        validation_set_data.append(preprocessing_input(file_path=filePath, file_name=filename, dictionary=gt, training=False))  # Set training=False for validation
t2 = datetime.datetime.utcnow()

# Measuring execution time
print('Elapsed time: ' + str(t2-t1))


Elapsed time: 0:24:53.158469


In [ ]:
validation_set_data



Output hidden; open in https://colab.research.google.com to view.

In [ ]:

with open(savename, "wb") as f:
    pickle.dump(validation_set_data, f)